In [14]:
import pandas as pd
import glob,os
import re


In [15]:
OMOP_FILES_PATH = 'codelists/OMOP/'

omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')
omop_relationships  = pd.read_csv(OMOP_FILES_PATH+'RELATIONSHIP.csv', sep='\t')
omop_concept_relationships  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT_RELATIONSHIP.csv', sep='\t')


omop_concepts

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1064190685.py:3: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,756315,metadata.metadata_type_concept_id,Metadata,CDM,Field,S,CDM1000,20210925,20991231,NaN
4,756316,metadata.name,Metadata,CDM,Field,S,CDM1001,20210925,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
1473553,33091,IQVIA CO-CONNECT,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473554,33092,IQVIA CO-CONNECT MIABIS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473555,33093,IQVIA CO-CONNECT TWINS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473556,33095,NHS Ethnic Category,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN


In [34]:
s='colorectal'
files = sorted( list( glob.glob("codelists/output/query_"+s+"*_snomed.csv") ) )
files


['codelists/output/query_glaucoma_0_snomed.csv',
 'codelists/output/query_glaucoma_1_snomed.csv',
 'codelists/output/query_glaucoma_2_snomed.csv',
 'codelists/output/query_glaucoma_3_snomed.csv']

In [35]:
for f in files:
    name = re.split('/|\.', f)[-2]
    input_snomed_list = pd.read_csv(f)

    input_snomed_list['conceptId'] = input_snomed_list['conceptId'].astype(str)
    omop_concepts['concept_code'] = omop_concepts['concept_code'].astype(str)
    omop_concepts['concept_id'] = omop_concepts['concept_id'].astype(str)

    omop_concepts_filtered = omop_concepts[ omop_concepts['concept_code'].isin(input_snomed_list['conceptId']) ]

    omop_concepts_filtered = omop_concepts_filtered[['concept_id','concept_name','vocabulary_id','concept_code']]
    omop_concepts_filtered

    #omop_relationships_filtered = omop_concept_relationships[ omop_concept_relationships['relationship_id'].isin(['Maps to','Subsumes'])]

    omop_concept_relationships['concept_id_1'] = omop_concept_relationships['concept_id_1'].astype(str)
    omop_concept_relationships['concept_id_2'] = omop_concept_relationships['concept_id_2'].astype(str)

    concept_maps = omop_concepts_filtered.merge( omop_concept_relationships, how='left', left_on='concept_id', right_on='concept_id_1' )
    concept_maps_named = concept_maps.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id' )

    target_codes_icd10 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD10PCS' ]
    target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
    print( target_codes_icd10['relationship_id'].value_counts() )

    target_codes_icd9 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD9Proc' ]
    target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
    print( target_codes_icd9['relationship_id'].value_counts() )

    target_codes_opcs4 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='OPCS4' ]
    target_codes_opcs4['opcs'] = target_codes_opcs4['concept_code_y'].str.replace('.','')

    print( target_codes_opcs4['relationship_id'].value_counts() )

    target_codes_icd9[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_icd9.csv')
    target_codes_icd10[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_icd10.csv')
    target_codes_opcs4[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_opcs.csv')

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:31: SettingWithCopyWarning: 
A value is tryin

relationship_id
Subsumes    14037
Name: count, dtype: int64
relationship_id
Mapped from    952
Subsumes       527
Name: count, dtype: int64
relationship_id
Mapped from    2267
Subsumes        483
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:31: SettingWithCopyWarning: 
A value is tryin

relationship_id
Subsumes    12000
Name: count, dtype: int64
relationship_id
Mapped from    733
Subsumes       451
Name: count, dtype: int64
relationship_id
Mapped from    1815
Subsumes        400
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:31: SettingWithCopyWarning: 
A value is tryin

relationship_id
Subsumes    150
Name: count, dtype: int64
relationship_id
Mapped from    95
Subsumes       42
Name: count, dtype: int64
relationship_id
Mapped from    177
Subsumes        27
Name: count, dtype: int64
relationship_id
Subsumes    140
Name: count, dtype: int64
relationship_id
Mapped from    65
Subsumes       38
Name: count, dtype: int64
relationship_id
Mapped from    155
Subsumes        27
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_43456/1421141509.py:31: SettingWithCopyWarning: 
A value is tryin

In [36]:
target_codes_opcs4[['concept_code_y','concept_name_y','opcs']].drop_duplicates()

,concept_code_y,concept_name_y,opcs
47,C57.3,Graft to sclera,C573
294,C01.8,Other specified excision of eye,C018
295,C01.9,Unspecified excision of eye,C019
297,C01,Excision of eye,C01
344,C47.8,Other specified closure of cornea,C478
...,...,...,...
7704,C73.4,Capsulotomy of lens NEC,C734
7705,C73.8,Other specified incision of capsule of lens,C738
7706,C73.9,Unspecified incision of capsule of lens,C739
7775,C79.2,Vitrectomy using pars plana approach,C792


In [37]:
target_codes_opcs4

,concept_id_x,concept_name_x,vocabulary_id_x,concept_code_x,concept_id_1,concept_id_2,relationship_id,valid_start_date_x,valid_end_date_x,invalid_reason_x,...,concept_name_y,domain_id,vocabulary_id_y,concept_class_id,standard_concept,concept_code_y,valid_start_date_y,valid_end_date_y,invalid_reason_y,opcs
47,4030789,Scleral reinforcement with graft,SNOMED,10807008,4030789,44509284,Mapped from,20180829,20991231,NaN,...,Graft to sclera,Procedure,OPCS4,Procedure,NaN,C57.3,19700101,20991231,NaN,C573
294,4003370,Eye excision,SNOMED,120086003,4003370,44508975,Subsumes,20180829,20991231,NaN,...,Other specified excision of eye,Procedure,OPCS4,Procedure,S,C01.8,19700101,20991231,NaN,C018
295,4003370,Eye excision,SNOMED,120086003,4003370,44508976,Mapped from,20180829,20991231,NaN,...,Unspecified excision of eye,Procedure,OPCS4,Procedure,NaN,C01.9,19700101,20991231,NaN,C019
297,4003370,Eye excision,SNOMED,120086003,4003370,46232214,Mapped from,20180829,20991231,NaN,...,Excision of eye,Procedure,OPCS4,Procedure,NaN,C01,20150401,20991231,NaN,C01
344,4004030,Cornea closure,SNOMED,120114008,4004030,44509239,Subsumes,20180829,20991231,NaN,...,Other specified closure of cornea,Procedure,OPCS4,Procedure,S,C47.8,19700101,20991231,NaN,C478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7704,4311822,Capsulotomy of lens,SNOMED,85785001,4311822,44508432,Mapped from,20180829,20991231,NaN,...,Capsulotomy of lens NEC,Procedure,OPCS4,Procedure,NaN,C73.4,19700101,20991231,NaN,C734
7705,4311822,Capsulotomy of lens,SNOMED,85785001,4311822,44508433,Subsumes,20180829,20991231,NaN,...,Other specified incision of capsule of lens,Procedure,OPCS4,Procedure,S,C73.8,19700101,20991231,NaN,C738
7706,4311822,Capsulotomy of lens,SNOMED,85785001,4311822,44508434,Mapped from,20180829,20991231,NaN,...,Unspecified incision of capsule of lens,Procedure,OPCS4,Procedure,NaN,C73.9,19700101,20991231,NaN,C739
7775,4314406,Mechanical vitrectomy by pars plana approach,SNOMED,87021001,4314406,44508455,Mapped from,20180829,20991231,NaN,...,Vitrectomy using pars plana approach,Procedure,OPCS4,Procedure,NaN,C79.2,19700101,20991231,NaN,C792


In [38]:
omop_concept_relationships[omop_concept_relationships['concept_id_1']=='4096461']

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
624715,4096461,46274039,Value mapped from,20220801,20991231,NaN
11186518,4096461,40642537,Has status,20220128,20991231,NaN
11186519,4096461,40642539,Has Module,20220128,20991231,NaN
14623737,4096461,2002767,Mapped from,20180625,20991231,NaN
14623738,4096461,2002768,Subsumes,20180625,20991231,NaN
14623739,4096461,2002769,Subsumes,20180625,20991231,NaN
14623740,4096461,2002770,Subsumes,20180625,20991231,NaN
14623741,4096461,4018020,Concept was_a from,20140401,20991231,NaN
14623742,4096461,4018020,Mapped from,19700101,20991231,NaN
14623743,4096461,4018161,Concept was_a from,20140401,20991231,NaN


In [39]:
method_relationships = omop_concept_relationships[omop_concept_relationships['relationship_id']=='Has method']
method_relationships

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
14106,4237501,4045073,Has method,20220128,20991231,NaN
20970,619843,40293773,Has method,20220128,20991231,NaN
97320,36674184,4044176,Has method,20190131,20991231,NaN
97337,36674186,4081836,Has method,20190131,20991231,NaN
97339,36674186,4106470,Has method,20190131,20991231,NaN
...,...,...,...,...,...,...
18892362,619793,4044525,Has method,20220128,20991231,NaN
18892369,619794,4044525,Has method,20220128,20991231,NaN
18892385,619795,40293775,Has method,20220128,20991231,NaN
18892400,618616,40293773,Has method,20220128,20991231,NaN


In [40]:
snomed2opcs = target_codes_opcs4[['concept_id_x','concept_name_x','concept_name_y','concept_id_y','concept_code_y']]

snomed2opcs

m1 = snomed2opcs.merge(method_relationships, how='left', left_on='concept_id_x', right_on='concept_id_1')
m1
m2 = m1.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id')
m2

methods = m2[['concept_name_x','concept_name_y','concept_code_y','concept_name']]

In [41]:
opcs2cat = pd.DataFrame(data={
    'opcs_name': methods['concept_name_y'],
    'opcs_code': methods['concept_code_y'],
    'category': methods['concept_name']
}).drop_duplicates()
opcs2cat

,opcs_name,opcs_code,category
0,Graft to sclera,C57.3,Reinforcement - action
1,Other specified excision of eye,C01.8,Excision - action
2,Unspecified excision of eye,C01.9,Excision - action
3,Excision of eye,C01,Excision - action
4,Other specified closure of cornea,C47.8,Closure - action
...,...,...,...
193,Capsulotomy of lens NEC,C73.4,Incision - action
194,Other specified incision of capsule of lens,C73.8,Incision - action
195,Unspecified incision of capsule of lens,C73.9,Incision - action
196,Vitrectomy using pars plana approach,C79.2,Excision - action


In [42]:
opcs2cat['opcs_code'].value_counts()

opcs_code
C54.3    3
C64.6    2
C54      2
C79.5    2
C69.4    2
        ..
C72.3    1
C73.2    1
C73.3    1
C74.3    1
C65.3    1
Name: count, Length: 182, dtype: int64